# 패키지 임포트

In [1]:
import random
import pandas as pd
import numpy as np
import tensorflow as tf
import nltk
import os
import string
import re
import json
import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Attention
from keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from transformer_build import  *
from data_preprocessing import *
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

# 데이터 로드

In [2]:
df = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/textdataset/WELFake/WELFake_Dataset.csv')

In [3]:
textcuboid=np.load('./1-Channel textcuboid_WELFake(elmo).npy')
textcuboid_test=np.load('./1-Channel textcuboid_test_WELFake(elmo).npy')

# 전처리

In [4]:
df = df.dropna()
df.isnull().sum()

df.drop(columns=['Unnamed: 0'],inplace=True)

df.drop_duplicates(inplace=True)

X = df.drop(columns=['label'])
y = df['label']

X[['title','text']] = X[['title','text']].applymap(lambda x:remove_punctuation(x))
X[['title','text']] = X[['title','text']].applymap(lambda x:x.lower())

X=X['title']+' '+X['text']

X = X.apply(clean_text)
X = list(X)

pattern = '[^a-z ]'
Clean_X=[]
for sen in X:
    Clean_X.append(re.sub(pattern, '', str(sen)))

clean_df = pd.DataFrame({'Clean_X': Clean_X, 'y': y})

fake_df = clean_df[clean_df['y'] == 0]
real_df = clean_df[clean_df['y'] == 1]

fake_x=list(fake_df['Clean_X'])
real_x=list(real_df['Clean_X'])

real_selected_lst = []
fake_selected_lst = []

for sen in real_x:
    word_count = len(sen.split())
    if 10 <= word_count < 2000:
        real_selected_lst.append(sen)
        
for sen in fake_x:
    word_count = len(sen.split())
    if 10 <= word_count < 2000:
        fake_selected_lst.append(sen)
        
X=real_selected_lst[:10000]+fake_selected_lst[:10000]
y=[0]*10000+[1]*10000

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
to_txt=x_train+x_test

In [5]:
encoder=LabelEncoder()

encoder.fit(y)

label=encoder.transform(y)

y_train=list(label[:16000])
y_test=list(label[16000:])

In [6]:
sos_x_train=[]
sos_x_test=[]
for sen in x_train:
    sos_x_train.append('<sos> '+sen)
for sen in x_test:
    sos_x_test.append('<sos> '+sen)

all_txt=sos_x_train+sos_x_test

tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_txt)

vocab_size =len(tokenizer.word_index)+1 #1을 더해야 에러가 안터짐 토큰 영향으로 보임

x_train_encoded = tokenizer.texts_to_sequences(sos_x_train)
x_test_encoded = tokenizer.texts_to_sequences(sos_x_test)

max_len = 1000

xtext_train = tf.keras.preprocessing.sequence.pad_sequences(x_train_encoded, maxlen=max_len)
xtext_test = tf.keras.preprocessing.sequence.pad_sequences(x_test_encoded, maxlen=max_len)

In [7]:
tmp = [[x,y,z] for x, y, z in zip(xtext_train, textcuboid, y_train)]
random.shuffle(tmp)
xtext_train = [n[0] for n in tmp]
textcuboid = [n[1] for n in tmp]
y_train = [n[2] for n in tmp]
xtext_train=np.array(xtext_train)
textcuboid=np.array(textcuboid)
y_train=np.array(y_train)

In [8]:
textcuboid_train=textcuboid[1000:]
text_train=xtext_train[1000:]
textcuboid__val=textcuboid[:1000]
text_val=xtext_train[:1000]
y_val=y_train[:1000]
y_train=y_train[1000:]

text_test=xtext_test
y_test=np.array(y_test)

# 모델 빌드

In [9]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model_{epoch}.h5", save_best_only=False, period=5
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=2, min_lr=0.0001
    ),
]

In [10]:
embedding_dim = 192  # 각 단어의 임베딩 벡터의 차원
num_heads = 1  # 어텐션 헤드의 수
dff = 128  # 포지션 와이즈 피드 포워드 신경망의 은닉층의 크기
num_transformer_blocks = 1 #트랜스포머 블록 적층

#ELMo channel의 학습
ELMo_input = Input(shape=(391, 256))
conv1 = Conv1D(1024, 1, padding='valid', activation='relu')(ELMo_input)
pooling = GlobalMaxPooling1D()(conv1)
ELMo_x = Dense(256, activation='relu')(pooling)
# ELMo_attention_layer = Attention()
# ELMo_a=ELMo_attention_layer([ELMo_x,ELMo_x])

#Sequence channel의 학습
Text_inputs = Input(shape=(max_len,))
embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, embedding_dim)
Text_x = embedding_layer(Text_inputs)

for _ in range(num_transformer_blocks):
    transformer_block = TransformerBlock(embedding_dim, num_heads, dff)
    Text_x = transformer_block(Text_x)

Text_x = tf.keras.layers.GlobalAveragePooling1D()(Text_x)
Text_x = tf.keras.layers.Dropout(0.5)(Text_x)
Text_x = tf.keras.layers.Dense(256, activation="relu")(Text_x)
# Text_attention_layer = Attention()
# Text_a=Text_attention_layer([Text_x,Text_x])

# 결합 후 분류
x = layers.concatenate([ELMo_x, Text_x], axis=-1)
# x = layers.concatenate([ELMo_x,ELMo_a, Text_x,Text_a], axis=-1)
# x = Dropout(0.5)(x)
output_layer = Dense(2, activation='softmax')(x)

model = Model(inputs=[ELMo_input, Text_inputs], outputs=output_layer)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 1000)]       0           []                               
                                                                                                  
 token_and_position_embedding (  (None, 1000, 192)   34284480    ['input_2[0][0]']                
 TokenAndPositionEmbedding)                                                                       
                                                                                                  
 input_1 (InputLayer)           [(None, 391, 256)]   0           []                               
                                                                                                  
 transformer_block (Transformer  (None, 1000, 192)   198464      ['token_and_position_embeddin

In [ ]:
history = model.fit([textcuboid_train,text_train], y_train,callbacks=callbacks, epochs=50,batch_size=256,validation_data=([textcuboid__val,text_val], y_val))

Epoch 1/15
35/59 [================>.............] - ETA: 3s - loss: 0.9959 - accuracy: 0.5532

In [ ]:
model.evaluate([textcuboid_test,text_test],y_test)

In [ ]:
from keras.models import load_model
custom_objects = {"TokenAndPositionEmbedding": TokenAndPositionEmbedding, "TransformerBlock": TransformerBlock}
model = load_model('best_model_10.h5', custom_objects=custom_objects)
model.evaluate([textcuboid_test,text_test],y_test)